## Esteban Mesa HW3: Chapter 4 & 5

Top 5 Queries:

### TOP 1 - Chapter 4: Exercise 6

- Write a query that returns customers who placed orders in 2015 but not in 2016:
- Tables involved: Sales.Customers and Sales.Orders


- EXPECTED OUTPUT:
| custid | companyname  |
|--------|--------------|
| 21     | Customer KIDPX |
| 23     | Customer F    |
| 33     | Customer FVXPQ |
| 36     | Customer LVJSO |
| 43     | Customer UISOJ |
| 51     | Customer PVDZC |
| 85     | Customer ENQZT |


In [ ]:
USE TSQLV4
SELECT DISTINCT C.custid, C.companyname
FROM Sales.Customers AS C
JOIN Sales.Orders AS O ON C.custid = O.custid
WHERE YEAR(O.orderdate) = 2015
AND C.custid NOT IN (
    SELECT O2.custid
    FROM Sales.Orders AS O2
    WHERE YEAR(O2.orderdate) = 2016
)
ORDER BY C.custid;

USE Northwinds2022TSQLV7
SELECT DISTINCT C.CustomerId, C.CustomerCompanyName
FROM Sales.Customer AS C
JOIN Sales.[Order] AS O ON C.CustomerId = O.CustomerId
WHERE YEAR(O.orderdate) = 2015
AND C.CustomerId NOT IN (
    SELECT O2.CustomerId
    FROM Sales.[Order] AS O2
    WHERE YEAR(O2.orderdate) = 2016
)
ORDER BY C.CustomerId;



Explanation: This query identifies customers who were active in 2015 but had no orders in 2016. It starts by connecting customer information in the Sales.Customers table with their order history in the Sales.Orders table.  Next, it focuses on orders specifically from the year 2015.  Then, using a clever subquery, it finds all customers who did place orders in 2016 within the Sales.Orders table and filters the main results to exclude those customers. Finally, it presents a list of distinct customer IDs and company names from the Sales.Customers table, ordered by customer ID.

### TOP 2 - Chapter 5: Exercise 4

- Write a solution using a recursive CTE that returns the management chain leading to Patricia Doyle (employee ID 9):
- Table involved: HR.Employees

EXPECTED OUTPUT:
|empid|mgrid|firstname|lastname|
|---|---|---|---|
|9|5|Patricia|Doyle|
|5|2|Sven|Mortensen|
|2|1|Don|Funk|
|1|NULL|Sara|Davis|


In [ ]:
USE TSQLV4;
GO

WITH RecursiveManagementChain AS (
    SELECT empid, mgrid, firstname, lastname
    FROM HR.Employees
    WHERE empid = 9
    UNION ALL
    SELECT e.empid, e.mgrid, e.firstname, e.lastname
    FROM HR.Employees e
    INNER JOIN RecursiveManagementChain rmc ON e.empid = rmc.mgrid
)
SELECT * FROM RecursiveManagementChain;

USE Northwinds2022TSQLV7;
GO

WITH RecursiveManagementChain AS (
    SELECT EmployeeId, EmployeeManagerId, EmployeeFirstName, EmployeeLastName
    FROM HumanResources.Employee
    WHERE EmployeeId = 9
    UNION ALL
    SELECT e.EmployeeId, e.EmployeeManagerId, e.EmployeeFirstName, e.EmployeeLastName
    FROM HumanResources.Employee e
    INNER JOIN RecursiveManagementChain rmc ON e.EmployeeId = rmc.EmployeeManagerId
)
SELECT * FROM RecursiveManagementChain;



Explanation: So basically, this query creates a hierarchical management chain starting from a specific employee, identified as Patricia Doyle with an employee ID of 9, and traces upwards through the organizational structure to the topmost manager. Utilizing a recursive Common Table Expression (CTE), it first selects Patricia Doyle's record from the HumanResources.Employee table, capturing essential details like the employee ID and name. The recursion then progressively joins each employee to their direct manager using the employee-managers relationship, effectively climbing the management ladder. This iterative process continues until it reaches the apex of the hierarchy, where a manager does not report to anyone else. The final output is a sequenced list that outlines the entire management chain from Patricia Doyle to the organization's highest authority, illustrating the direct and indirect reporting lines within the company. 

## PROPOSITIONS:

### TOP 3 - Chapter 4.Q1
- Proposition: Retrieve order IDs for employees whose last name starts with 'D'.
- Use Table: Sales.Orders and HR.Employees

In [ ]:
--USE TSQLV4
--SELECT orderid
--FROM Sales.Orders
--WHERE empid IN
--  (SELECT E.empid
--   FROM HR.Employees AS E
--   WHERE E.lastname LIKE N'D%');
--GO

USE Northwinds2022TSQLV7;
SELECT OrderId
FROM Sales.[Order]
WHERE EmployeeID IN
  (SELECT E.EmployeeID
   FROM HumanResources.Employee AS E
   WHERE E.EmployeeLastName LIKE N'D%'); 
GO



- Explanation: This query selects all order IDs from the "Sales.Orders" table where the corresponding employee ID matches an employee whose last name begins with the letter 'D' in the "HR.Employees" table.

### TOP 4 - Chapter 4.Q2:
- Proposition: List customers without orders.
- Use Table: Sales.Customers and Sales.Orders

In [ ]:
-- Following returns an empty set
--USE TSQLV4
--SELECT custid, companyname
--FROM Sales.Customers
--WHERE custid NOT IN(SELECT O.custid
--                    FROM Sales.Orders AS O);


USE Northwinds2022TSQLV7
SELECT CustomerId, CustomerCompanyName
FROM Sales.[Customer]
WHERE CustomerId NOT IN(SELECT O.CustomerId
                    FROM Sales.[order] AS O);

- Explanation: This query fetches customer IDs and company names from the "Sales.Customers" table for those customers who have not placed any orders, as indicated by the absence of their customer ID in the "Sales.Orders" table.

### TOP 5 - Chapter 4.Q3
- Proposition: Calculate the percentage of each order's value to the total orders per customer.
- Use Table: Sales.OrderValues

In [ ]:
-- Percentage of customer total
--USE TSQLV4
--GO
--SELECT orderid, custid, val,
--  CAST(100. * val / (SELECT SUM(O2.val)
--                     FROM Sales.OrderValues AS O2
--                     WHERE O2.custid = O1.custid)
--       AS NUMERIC(5,2)) AS pct
--FROM Sales.OrderValues AS O1
--ORDER BY custid, orderid;

-------------------------------
USE Northwinds2022TSQLV7;
GO
WITH OrderValues AS (
    SELECT 
        od.OrderId, 
        o.CustomerId, 
        SUM(od.UnitPrice * od.Quantity * (1 - od.DiscountPercentage / 100.0)) AS val
    FROM 
        Sales.[OrderDetail] od
    JOIN 
        Sales.[Order] o ON od.OrderId = o.OrderId
    GROUP BY 
        od.OrderId, 
        o.CustomerId
)
SELECT 
    O1.OrderId, 
    O1.CustomerId, 
    CAST(100.0 * O1.val / (SELECT SUM(O2.val)
                           FROM OrderValues AS O2
                           WHERE O2.CustomerId = O1.CustomerId)
         AS NUMERIC(5,2)) AS pct
FROM 
    OrderValues AS O1
ORDER BY 
    O1.CustomerId, O1.OrderId;


Explanation: This query selects order IDs, customer IDs, and the value of each order as a percentage of the total value of all orders placed by the same customer, formatted as a numeric value with two decimal places.

Chapter 4.Q4:
- Proposition: Display yearly order quantities and running totals.
- Use Table: Sales.OrderTotalsByYear

In [ ]:
--USE TSQLV4
--SELECT orderyear, qty
--FROM Sales.OrderTotalsByYear;

--SELECT orderyear, qty,
--  (SELECT SUM(O2.qty)
--   FROM Sales.OrderTotalsByYear AS O2
--   WHERE O2.orderyear <= O1.orderyear) AS runqty
--FROM Sales.OrderTotalsByYear AS O1
--ORDER BY orderyear;

USE Northwinds2022TSQLV7;
GO
WITH YearlyOrderCounts AS (
    SELECT 
        YEAR(OrderDate) AS OrderYear, 
        COUNT(*) AS Qty
    FROM 
        Sales.[Order]
    GROUP BY 
        YEAR(OrderDate)
)
SELECT 
    OrderYear, 
    Qty
FROM 
    YearlyOrderCounts
ORDER BY 
    OrderYear;
WITH YearlyOrderCounts AS (
    SELECT 
        YEAR(OrderDate) AS OrderYear, 
        COUNT(*) AS Qty
    FROM 
        Sales.[Order]
    GROUP BY 
        YEAR(OrderDate)
),
RunningOrderTotals AS (
    SELECT 
        Y.OrderYear,
        Y.Qty,
        (SELECT SUM(Y2.Qty) FROM YearlyOrderCounts AS Y2 WHERE Y2.OrderYear <= Y.OrderYear) AS RunQty
    FROM 
        YearlyOrderCounts AS Y
)
SELECT 
    OrderYear, 
    Qty,
    RunQty
FROM 
    RunningOrderTotals
ORDER BY 
    OrderYear;




- Explanation: This set of queries first selects the total quantity of orders per year and then, for each year, calculates the cumulative sum (running total) of orders up to and including that year.

Chapter 4.Q5:
- Proposition: Create and populate the 'MyShippers' table.
- Use Table: Creating and populating Sales.MyShippers

In [ ]:
-- Create and populate table Sales.MyShippers
--USE TSQLV4
--DROP TABLE IF EXISTS Sales.MyShippers;

--CREATE TABLE Sales.MyShippers
--(
--  shipper_id  INT          NOT NULL,
--  companyname NVARCHAR(40) NOT NULL,
--  phone       NVARCHAR(24) NOT NULL,
--  CONSTRAINT PK_MyShippers PRIMARY KEY(shipper_id)
--);

--INSERT INTO Sales.MyShippers(shipper_id, companyname, phone)
--  VALUES(1, N'Shipper GVSUA', N'(503) 555-0137'),
--          (2, N'Shipper ETYNR', N'(425) 555-0136'),
--                (3, N'Shipper ZHISN', N'(415) 555-0138');

USE Northwinds2022TSQLV7;
GO

DROP TABLE IF EXISTS Sales.MyShippers;
GO

CREATE TABLE Sales.MyShippers
(
  shipper_id  INT          NOT NULL,
  companyname NVARCHAR(40) NOT NULL,
  phone       NVARCHAR(24) NOT NULL,
  CONSTRAINT PK_MyShippers PRIMARY KEY(shipper_id)
);
GO

INSERT INTO Sales.MyShippers(shipper_id, companyname, phone)
VALUES
  (1, N'Shipper GVSUA', N'(503) 555-0137'),
  (2, N'Shipper ETYNR', N'(425) 555-0136'),
  (3, N'Shipper ZHISN', N'(415) 555-0138');
GO




- Explanation: This query drops the "Sales.MyShippers" table if it exists, creates a new one, and inserts three records into it, defining columns for shipper ID, company name, and phone number.

Chapter 5.Q1
- Proposition: Get the three most recent orders for each customer.
- Use Table: Sales.Customers and Sales.Orders with OUTER APPLY

In [ ]:
-- 3 most recent orders for each customer, preserve customers
--USE TSQLV4
--SELECT C.custid, A.orderid, A.orderdate
--FROM Sales.Customers AS C
--  OUTER APPLY
--    (SELECT TOP (3) orderid, empid, orderdate, requireddate 
--     FROM Sales.Orders AS O
--     WHERE O.custid = C.custid
--     ORDER BY orderdate DESC, orderid DESC) AS A;

USE Northwinds2022TSQLV7
SELECT C.CustomerId, A.OrderId, A.OrderDate
FROM Sales.[Customer] AS C
  OUTER APPLY
    (SELECT TOP (3) OrderId, employeeId, OrderDate, RequiredDate
     FROM Sales.[Order] AS O
     WHERE O.CustomerId = C.CustomerId
     ORDER BY OrderDate DESC, OrderId DESC) AS A;

- Explanation: This query employs an OUTER APPLY to retrieve the top three most recent orders for each customer from the "Sales.Orders" table, ensuring even customers without orders are listed.

Chapter 5.Q2:
- Proposition: Generate a cross-product of shippers and employees.
- Use Table: Sales.Shippers and HR.Employees with CROSS APPLY

In [ ]:
--USE TSQLV4
--SELECT S.shipperid, E.empid FROM Sales.Shippers AS S CROSS APPLY HR.Employees AS E;

USE Northwinds2022TSQLV7
SELECT S.ShipperId, E.EmployeeId FROM Sales.[Shipper] AS S CROSS APPLY HumanResources.Employee AS E;

- Explanation: This query uses CROSS APPLY to combine each shipper with each employee, resulting in a table that pairs every shipper with every employee.

Chapter 5.Q3
- Proposition: Create a view for USA customers and test schema changes.
- Use Table: Creating view Sales.USACusts based on Sales.Customers

In [ ]:
--USE TSQLV4
--GO
--ALTER VIEW Sales.USACusts
--WITH SCHEMABINDING
--AS
--SELECT custid, companyname, contactname, contacttitle, address, city, region, postalcode, country, phone, fax
--FROM Sales.Customers
--WHERE country = N'USA';
--GO

USE Northwinds2022TSQLV7;
GO
ALTER VIEW Sales.[USACusts]
WITH SCHEMABINDING
AS
SELECT 
    CustomerID, 
    CustomerCompanyName, 
    CustomerContactName, 
    CustomerAddress, 
    CustomerCity, 
    CustomerRegion, 
    CustomerPostalCode, 
    CustomerCountry, 
    CustomerPhoneNumber, 
    CustomerFaxNumber
FROM 
    Sales.[Customer]
WHERE 
    CustomerCountry = N'USA';
GO

- Explanation: This query creates a view named "Sales.USACusts" that includes customers from the USA and attempts to make a schema change to the "Sales.Customers" table by dropping the 'address' column (commented out).

Chapter 5.Q4
- Proposition: Count unique customers per year.
- Use Table: Sales.Orders

In [ ]:
--USE TSQLV4
--GO
--WITH C AS ( SELECT YEAR(orderdate) AS orderyear, custid FROM Sales.Orders ) 
--SELECT orderyear, COUNT(DISTINCT custid) AS numcusts 
--FROM C GROUP BY orderyear;
--GO
USE Northwinds2022TSQLV7
GO
WITH C AS ( SELECT YEAR(OrderDate) AS orderyear, CustomerId FROM Sales.[Order] ) 
SELECT orderyear, COUNT(DISTINCT CustomerId) AS NumCusts
FROM C GROUP BY orderyear;
GO

- Explanation: This query calculates the number of distinct customers for each year based on the order date from the "Sales.Orders" table, grouping the results by year.

Chapter 5.Q5
- Proposition: Select USA customers using a subquery.
- Use Table: Sales.Customers

In [ ]:
--USE TSQLV4
--SELECT * FROM (SELECT custid, companyname FROM Sales.Customers WHERE country = N'USA') AS USACusts;

USE Northwinds2022TSQLV7
SELECT * FROM (SELECT CustomerId, CustomerCompanyName FROM Sales.[Customer] WHERE CustomerCountry = N'USA') AS USACusts;


- Explanation: This query selects all customers from the "Sales.Customers" table who are located in the USA, using a subquery to create a temporary table named "USACusts" containing their customer ID and company name.